In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification')
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification/src')

In [ ]:
from llm_lexical_simplifier import LLMLexicalSimplifier
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForMaskedLM
from simple_bert_lexical_simplifier import SimpleBertLexicalSimplifier
from benchmark_suite import BenchmarkSuite
from language import Language

# LLM based

In [ ]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    torch_dtype="auto",
    trust_remote_code=True,
    # Flash attention is only supported starting Ampere architecture
    # So we cant use it on free-tier GPUs
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

In [ ]:
generation_args = {
    "max_new_tokens": 100,
    "return_full_text": False,
    "do_sample": False,
}

# Define PAD Token = EOS Token
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
de_messages = [
    {"role": "user", "content": "Der Maschinenbau hat in Europa durch die Bildung der EU eine starke Erleichterung erhalten. Die vereinfachte Version des vorigen Satzes ist: Der Maschinenbau hat in Europa durch die Bildung der EU eine starke [MASK] erhalten. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["Vereinfachung", "Entspannung", "Begünstigung", "Unterstützung", "Rückenwind,  "Förderung", "Rückhalt", "Aufwind", "Antrieb", "Erleichterung"]'},
    {"role": "user", "content": "Die EU-Renaturierungsverordnung galt eigentlich bereits als durchgebracht; das EU-Parlament hatte zugestimmt und die Staaten ihren grundsätzlichen Sanktus signalisiert. Die vereinfachte Version des vorigen Satzes ist: Die EU-Renaturierungsverordnung galt eigentlich bereits als durchgebracht; das EU-Parlament hatte zugestimmt und die Staaten ihren grundsätzlichen [MASK] signalisiert. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["Zustimmung", "Einverständnis", "Okay", "Segen", "Billigung, Bestätigung, Bejahung, Zuspruch, Akzeptanz, Befürwortung"]'},
]
en_messages = [
    {"role": "user", "content": "The president’s relatives were also said to be critical of the way his closest advisers had prepared him for the debate. The simplified version of the previous sentence is: The president’s relatives were also said to be critical of the way his closest [MASK] had prepared him for the debate. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["staff", "team", "aides", "consultants", "assistants", "strategists", "supporters", "handlers", "counselors", "staffers"]'},
    {"role": "user", "content": "The intermediate sprint will come 94km or so into the stage. The simplified version of the previous sentence is: The intermediate sprint will come 94km or so into the [MASK]. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["segment", "part", "section", "phase", "leg", "interval", "course", "route", "portion", "strech"]'},
]

llm_ls = LLMLexicalSimplifier(model=model, tokenizer=tokenizer,
                              pattern='{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}. Ten results as ordered, AST parsable Python list:',
                              exemplars=en_messages, mask_token='[MASK]', generation_args=generation_args)

In [ ]:
llm_output = llm_ls.generate_substitutions_for('heikel', 'Die politische Lage in Syrien ist heikel.')
llm_output

In [ ]:
llm_output

In [ ]:
suite = BenchmarkSuite(llm_ls, {
        # Just run DE for now since EN has way more datasets and takes longer
        #Language.EN: {'pattern': '{original_sentence}. The simplified version of the previous sentence is: '
        #                         '{sentence_with_complex_word_masked}. Ten results as ordered, AST parsable '
        #                         'Python list:',
        #              'exemplars': en_messages},
        Language.DE: {'pattern': '{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: '
                                 '{sentence_with_complex_word_masked}. Ten results as ordered, AST parsable '
                                 'Python list:',
                      'exemplars': de_messages}})
suite.run()

#BERT based

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
bert_model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-cased")

In [ ]:
bert_ls = SimpleBertLexicalSimplifier(bert_model, bert_tokenizer, '{original_sentence}. The simplified version of the previous sentence is: {sentence_with_complex_word_masked}. Top ten results:', None, '[MASK]')

In [ ]:
suite = BenchmarkSuite(bert_ls,
                       {Language.EN: {'pattern': '{original_sentence}. The simpler version of the previous sentence is: {sentence_with_complex_word_masked}. Top ten results:', 'exemplars': None},
                        Language.DE: {'pattern': '{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}. Die zehn besten Resultate:', 'exemplars': None},
                        Language.ES: {'pattern': '{original_sentence}. La versión simplificada de la frase anterior es: {sentence_with_complex_word_masked}. Los diez primeros resultados:', 'exemplars': None},
                        Language.PT: {'pattern': '{original_sentence}. A versão simplificada da frase anterior é: {sentence_with_complex_word_masked}. Os dez melhores resultados:', 'exemplars': None}})
suite.run()